In [ ]:
!pipenv install pdfplumber

In [1]:
import pandas as pd
from geopy.geocoders import Bing
from geopy.geocoders import Nominatim
import os
import time
import re
import json

In [2]:
import pdfplumber
import pandas as pd

# Open the PDF file
with pdfplumber.open("Toelichting-variabelen-kwb-2023.pdf") as pdf:
    all_tables = []
    # Iterate through each page
    for page in pdf.pages:
        # Extract tables from the page
        tables = page.extract_tables()
        # Append the tables to the list of all tables
        all_tables.extend(tables)

# Convert each table to a DataFrame and concatenate them
dfs = [pd.DataFrame(table) for table in all_tables]
concatenated_df = pd.concat(dfs, ignore_index=True)

# Keep only the first column
concatenated_df = concatenated_df.iloc[:, [0]]

# Use the first row as header
concatenated_df.columns = concatenated_df.iloc[0]

# Remove the first row (since it's now the header)
concatenated_df = concatenated_df.iloc[1:]

# Rename columns
concatenated_df.columns = ["variabele"]

# Split the column on the colon ":" delimiter
concatenated_df["omschrijving"] = concatenated_df["variabele"].str.split(":", n=1).str[-1]
concatenated_df["variabele"] = concatenated_df["variabele"].str.split(":", n=1).str[0]

# Remove records where "omschrijving" is missing
concatenated_df = concatenated_df.dropna(subset=["omschrijving"])

# Reset index
concatenated_df.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
print(concatenated_df)


         variabele                           omschrijving
0            regio                        Regioaanduiding
1          gm_naam                           Gemeentenaam
2             recs                            Soort regio
3         gwb_code                               Codering
4          ind_wbi   Indelingswijziging wijken en buurten
..             ...                                    ...
128        pst_mvp             Meest voorkomende postcode
129       pst_dekp                     Dekkingspercentage
130  Stedelijkheid                          Stedelijkheid
131        ste_mvs                 Mate van stedelijkheid
132        ste_oad             Omgevingsadressendichtheid

[133 rows x 2 columns]


In [11]:
concatenated_df.to_csv("omschrijvingen KWB.csv", sep=";", index=False)

In [10]:
# Sort the DataFrame by the length of the "omschrijving" column in descending order
sorted_df = concatenated_df.sort_values(by="omschrijving", key=lambda x: x.str.len(), ascending=False)

# Get the 5 longest descriptions
top_5_longest = sorted_df.head(5)

# Print the top 5 longest descriptions
print(top_5_longest)


    variabele                                  omschrijving
97    p_jz_tn   Percentage jongeren met jeugdzorg in natura
87   p_hh_110      Huishoudens tot 110% van sociaal minimum
109  a_bed_ru      R-U Cultuur, recreatie, overige diensten
88   p_hh_120      Huishoudens tot 120% van sociaal minimum
96    a_jz_tn       Aantal jongeren met jeugdzorg in natura
